# A map of World Cup stadia using wikidata

[Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) is an amazing project 
that aims to turn the unstructured text of  Wikipedia into a database of facts 
and figures that allows you to go beyond just presenting a page about something 
to using data about it.

I've been wanting to try out using it, and "SPARQL", the language used to query
it, so I decided to try and create a map of every stadium that has hosted a game
at the Fifa World Cup finals - a topical query as the 2022 World Cup is midway through.

## Step 1. Querying the data

I used [query.wikidata.org](https://query.wikidata.org/) to come up with a query
that got me the data I was looking for. Having never used SPARQL before it took 
a bit of tweaking to get the query I needed - I found the interface helpful for
finding the right entities and the included examples for how to structure it.

Here's the query I came up with. I'll go through what each part does below.

In [1]:
wc_sparql = """
SELECT ?competitionLabel ?date ?teamLabel ?countryLabel ?capitalLabel ?coords WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?competition wdt:P3450 ?competition_ids.
  FILTER (?competition_ids IN (wd:Q19317, wd:Q260858 ) ).
  ?competition wdt:P582 ?date.
  ?competition wdt:P1346 ?team.
  ?team wdt:P1532 ?country.
  ?country wdt:P36 ?capital.
  ?capital wdt:P625 ?coords
}
ORDER BY ?date
"""

The first part sets up the fields we want to return - the name of the World Cup, the location ID (a stadium), the name of the stadium, the latitude and longitude and the name of the country

```
SELECT ?FIFA_World_CupLabel ?location ?locationLabel ?coord ?countryLabel WHERE {
```

This next part allows you to fetch labels for each of the items, which is more helpful than the URI that gets returned.

```
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
```

Then we start off by adding a field called "FIFA_World_Cup" based on finding "sports season of league or competition" (`wdt:P3450`) with the labels "FIFA World Cup" (`wd:Q19317`)

```
?FIFA_World_Cup wdt:P3450 wd:Q19317.
```

Then we look for the locations (`wdt:P276`) attached to each of these competitions:

```
?FIFA_World_Cup wdt:P276 ?location.
```

And for each location we want the co-ordinates (`wdt:P625`) and country (`wdt:P17`).

```
?location wdt:P625 ?coord.
?location wdt:P17 ?country
```

I then used a python library called [SPARQLWrapper](https://rdflib.github.io/sparqlwrapper/) to send the query to the WikiData sparql endpoint, and get JSON data back.

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(wc_sparql)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

Here's an example of what one of the results looks like - a stadium used in the very first World Cup in Uruguay.

In [4]:
results['results']['bindings'][0]

{'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
  'type': 'literal',
  'value': '1930-07-30T00:00:00Z'},
 'coords': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
  'type': 'literal',
  'value': 'Point(-56.166666666 -34.866666666)'},
 'competitionLabel': {'xml:lang': 'en',
  'type': 'literal',
  'value': '1930 FIFA World Cup'},
 'teamLabel': {'xml:lang': 'en',
  'type': 'literal',
  'value': 'Uruguay national football team'},
 'countryLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Uruguay'},
 'capitalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Montevideo'}}

## Step 2: processing the results

I then want to turn the results into nicely formatted data for plotting on a map. I'm looking for data that contains
one record for each stadium, even if it has hosted games at more than one World Cup (e.g Mexico in 1970 and 1986). 

The co-ordinates for each location come in WKT format, so I use a library called [Shapely](https://pypi.org/project/Shapely/) to extract the latitude and longitude.

In [5]:
# for converting coordinates
import shapely.wkt
from datetime import date
import pandas as pd

Then I go through each of the results and add to a python dictionary. If the stadium is already in the dictionary I just add the extra World Cup year to the dictionary, rather than adding a new record.

In [6]:
def get_value(v):
    if v.get("datatype") == 'http://www.opengis.net/ont/geosparql#wktLiteral':
        return shapely.wkt.loads(v.get("value")).coords[0]
    if v.get("datatype") == "http://www.w3.org/2001/XMLSchema#dateTime":
        return date.fromisoformat(v.get("value")[:10])
    return v.get("value")

winners = pd.DataFrame({
        k: get_value(v)
        for k, v in result.items()
    } for result in results["results"]["bindings"])
winners

,date,coords,competitionLabel,teamLabel,countryLabel,capitalLabel
0,1930-07-30,"(-56.166666666, -34.866666666)",1930 FIFA World Cup,Uruguay national football team,Uruguay,Montevideo
1,1934-06-10,"(12.482777777, 41.893055555)",1934 FIFA World Cup,Italy national association football team,Italy,Rome
2,1938-06-19,"(12.482777777, 41.893055555)",1938 FIFA World Cup,Italy national association football team,Italy,Rome
3,1950-07-16,"(-56.166666666, -34.866666666)",1950 FIFA World Cup,Uruguay national football team,Uruguay,Montevideo
4,1954-07-04,"(13.383333333, 52.516666666)",1954 FIFA World Cup,Germany national association football team,Germany,Berlin
5,1958-06-29,"(-47.882777777, -15.793888888)",1958 FIFA World Cup,Brazil national football team,Brazil,Brasília
6,1960-07-10,"(37.617777777, 55.755833333)",1960 European Nations' Cup,Soviet Union national association football team,Soviet Union,Moscow
7,1962-06-17,"(-47.882777777, -15.793888888)",1962 FIFA World Cup,Brazil national football team,Brazil,Brasília
8,1964-06-21,"(-3.7025, 40.416666666)",1964 European Challenge Cup,Spain national association football team,Spain,Madrid
9,1966-07-30,"(-0.1275, 51.507222222)",1966 FIFA World Cup,England national association football team,England,London


Here's what an entry in the processed data looks like. I've used the wikidata URI as an identifier for each stadium.

In [7]:
england = winners[winners["countryLabel"]=="England"].iloc[0]
england

date                                                1966-07-30
coords                                 (-0.1275, 51.507222222)
competitionLabel                           1966 FIFA World Cup
teamLabel           England national association football team
countryLabel                                           England
capitalLabel                                            London
Name: 9, dtype: object

In [8]:
england["date"]

datetime.date(1966, 7, 30)

In [9]:
winners = winners[winners["date"]>england["date"]]
winners

,date,coords,competitionLabel,teamLabel,countryLabel,capitalLabel
10,1968-06-10,"(12.482777777, 41.893055555)",1968 UEFA European Championship,Italy national association football team,Italy,Rome
11,1970-06-21,"(-47.882777777, -15.793888888)",1970 FIFA World Cup,Brazil national football team,Brazil,Brasília
12,1972-06-18,"(13.383333333, 52.516666666)",UEFA Euro 1972,Germany national association football team,Germany,Berlin
13,1974-07-07,"(13.383333333, 52.516666666)",1974 FIFA World Cup,Germany national association football team,Germany,Berlin
14,1976-06-20,"(14.421388888, 50.0875)",UEFA Euro 1976,Czechoslovakia national association football team,Czechoslovakia,Prague
15,1978-06-25,"(-58.381944444, -34.599722222)",1978 FIFA World Cup,Argentina national football team,Argentina,Buenos Aires
16,1980-06-22,"(13.383333333, 52.516666666)",UEFA Euro 1980,Germany national association football team,Germany,Berlin
17,1982-07-11,"(12.482777777, 41.893055555)",1982 FIFA World Cup,Italy national association football team,Italy,Rome
18,1984-06-27,"(2.351388888, 48.856944444)",UEFA Euro 1984,France national association football team,France,Paris
19,1986-06-29,"(-58.381944444, -34.599722222)",1986 FIFA World Cup,Argentina national football team,Argentina,Buenos Aires


In [12]:
import haversine as hs

In [13]:
winners.loc[:, "Distance from home"] = winners['coords'].apply(lambda x: hs.haversine(x, england['coords'], unit=hs.Unit.MILES))
winners

c:\Users\drkan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,date,coords,competitionLabel,teamLabel,countryLabel,capitalLabel,Distance from home
10,1968-06-10,"(12.482777777, 41.893055555)",1968 UEFA European Championship,Italy national association football team,Italy,Rome,1092.454435
11,1970-06-21,"(-47.882777777, -15.793888888)",1970 FIFA World Cup,Brazil national football team,Brazil,Brasília,5175.337991
12,1972-06-18,"(13.383333333, 52.516666666)",UEFA Euro 1972,Germany national association football team,Germany,Berlin,936.064550
13,1974-07-07,"(13.383333333, 52.516666666)",1974 FIFA World Cup,Germany national association football team,Germany,Berlin,936.064550
14,1976-06-20,"(14.421388888, 50.0875)",UEFA Euro 1976,Czechoslovakia national association football team,Czechoslovakia,Prague,1009.907100
15,1978-06-25,"(-58.381944444, -34.599722222)",1978 FIFA World Cup,Argentina national football team,Argentina,Buenos Aires,6069.963766
16,1980-06-22,"(13.383333333, 52.516666666)",UEFA Euro 1980,Germany national association football team,Germany,Berlin,936.064550
17,1982-07-11,"(12.482777777, 41.893055555)",1982 FIFA World Cup,Italy national association football team,Italy,Rome,1092.454435
18,1984-06-27,"(2.351388888, 48.856944444)",UEFA Euro 1984,France national association football team,France,Paris,250.696980
19,1986-06-29,"(-58.381944444, -34.599722222)",1986 FIFA World Cup,Argentina national football team,Argentina,Buenos Aires,6069.963766


In [14]:
winners.loc[:, "Previous coords"] = winners.coords.shift(1).apply(lambda x: england["coords"] if pd.isna(x) else x)
winners

c:\Users\drkan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,date,coords,competitionLabel,teamLabel,countryLabel,capitalLabel,Distance from home,Previous coords
10,1968-06-10,"(12.482777777, 41.893055555)",1968 UEFA European Championship,Italy national association football team,Italy,Rome,1092.454435,"(-0.1275, 51.507222222)"
11,1970-06-21,"(-47.882777777, -15.793888888)",1970 FIFA World Cup,Brazil national football team,Brazil,Brasília,5175.337991,"(12.482777777, 41.893055555)"
12,1972-06-18,"(13.383333333, 52.516666666)",UEFA Euro 1972,Germany national association football team,Germany,Berlin,936.064550,"(-47.882777777, -15.793888888)"
13,1974-07-07,"(13.383333333, 52.516666666)",1974 FIFA World Cup,Germany national association football team,Germany,Berlin,936.064550,"(13.383333333, 52.516666666)"
14,1976-06-20,"(14.421388888, 50.0875)",UEFA Euro 1976,Czechoslovakia national association football team,Czechoslovakia,Prague,1009.907100,"(13.383333333, 52.516666666)"
15,1978-06-25,"(-58.381944444, -34.599722222)",1978 FIFA World Cup,Argentina national football team,Argentina,Buenos Aires,6069.963766,"(14.421388888, 50.0875)"
16,1980-06-22,"(13.383333333, 52.516666666)",UEFA Euro 1980,Germany national association football team,Germany,Berlin,936.064550,"(-58.381944444, -34.599722222)"
17,1982-07-11,"(12.482777777, 41.893055555)",1982 FIFA World Cup,Italy national association football team,Italy,Rome,1092.454435,"(13.383333333, 52.516666666)"
18,1984-06-27,"(2.351388888, 48.856944444)",UEFA Euro 1984,France national association football team,France,Paris,250.696980,"(12.482777777, 41.893055555)"
19,1986-06-29,"(-58.381944444, -34.599722222)",1986 FIFA World Cup,Argentina national football team,Argentina,Buenos Aires,6069.963766,"(2.351388888, 48.856944444)"


In [15]:
winners.apply(lambda x: hs.haversine(x['coords'], x['Previous coords'], unit=hs.Unit.MILES), axis=1).cumsum()

10     1092.454435
11     6555.361699
12    12498.691220
13    12498.691220
14    12676.699434
15    19551.585164
16    26452.088964
17    27170.131103
18    28016.915879
19    34137.322177
20    40534.154239
21    41120.820136
22    41340.856396
23    47374.697043
24    53318.026565
25    54120.234706
26    54120.234706
27    59311.937764
28    65275.585908
29    66093.696770
30    67216.577470
31    67216.577470
32    67216.577470
33    68659.598692
34    70481.361345
35    71539.009107
36    72385.793883
dtype: float64

## Step 3: Mapping the results

I really like [folium](http://python-visualization.github.io/folium/index.html) for easily
producing Leaflet-based maps in python. I'm going to also use the MarkerCluster plugin
to cluster the markers to make it easier to view all the stadia on one map - with clusters
based on countries.

In [16]:
import folium
from folium.plugins import MarkerCluster
import html

First I initialise the map and zoom out so you can see the whole world.

In [17]:
m = folium.Map(
    location=[20,0],
    zoom_start=2,
    tiles='Stamen Toner',
    attr='''<a id="home-link" target="_top" href="../">Map tiles</a> by 
    <a target="_top" href="http://stamen.com">Stamen Design</a>, 
    under <a target="_top" href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. 
    Data by <a target="_top" href="http://openstreetmap.org">OpenStreetMap</a>, 
    under <a target="_top" href="http://creativecommons.org/licenses/by-sa/3.0">CC BY SA</a>.
    | Locations powered by <a href="https://query.wikidata.org/">Wikidata</a>.'''
)

Then we go through the stadia and add each one to a cluster based on its country. I've also added a little popup which tells you the stadium's name and which World Cups it hosted games at. I also set a football icon for the pins.

In [18]:
clusters = {}
for stadium_id in stadia:
    
    s = stadia[stadium_id]
    
    if s["country"] not in clusters:
        clusters[s["country"]] = MarkerCluster().add_to(m)
    
    folium.Marker(
        [s["lat_lng"][1], s["lat_lng"][0]], 
        popup='{}, {} - <i>{}</i>'.format(
            html.escape(s["stadium"]), 
            html.escape(s["country"]),
            html.escape(", ".join(s["worldcups"]))
        ),
        icon=folium.Icon(icon='soccer-ball-o', prefix='fa')
    ).add_to(clusters[s["country"]])

NameError: name 'stadia' is not defined

Finally we show the resulting map, which can be zoomed and panned to look at particular countries.

In [89]:
m

In [83]:
m.save("world-cup-stadia-map.html")

As an extra I wanted to convert the data into GeoJSON format so it's easy to use elsewhere.

 - [world_cup_stadia.geojson](https://dkane.net/data/world_cup_stadia.geojson)

In [72]:
from geojson import Feature, Point, FeatureCollection

In [80]:
wc_geojson = FeatureCollection(
    [Feature(geometry=Point(stadia[s]["lat_lng"]), 
             properties=stadia[s]) for s in stadia]
)

In [78]:
with open('world_cup_stadia.geojson', 'w') as a:
    geojson.dump(wc_geojson, a, indent=4)

## Step 4: taking it further

This was just a quick exercise to try and get data out of wikidata and then use it. There's a few things that could be done to take it further:

- add filters to the map to filter by country, World Cup, etc.
- see if Wikidata has data on the matches that took place at each location and the teams that have played there, allowing you to filter by team or stage of the competition.
- visualise the data by adding in details like the maximum attendance 

## Acknowledgements

- The data is from Wikidata and is used under the [CC0 Public Domain licence](https://creativecommons.org/about/cc0). 
- [sparqlwrapper](https://rdflib.github.io/sparqlwrapper/) for querying the data.
- [Folium](http://python-visualization.github.io/folium/index.html) mapping using [Leaflet](https://leafletjs.com/) and [FontAwesome](http://fontawesome.com/icons?d=gallery&c=sports&m=free) icons.
- Map tiles by [Stamen Design](http://stamen.com), under [CC BY 3.0](http://creativecommons.org/licenses/by/3.0). Data by [OpenStreetMap](http://openstreetmap.org), under [CC BY SA](http://creativecommons.org/licenses/by-sa/3.0).
- [Shapely](https://pypi.org/project/Shapely/) and [geojson](https://pypi.org/project/geojson/) python libraries for manipulating the geodata
- [This blog post was written alongside the code in a Juypter notebook](https://nbviewer.jupyter.org/urls/dkane.net/data/world_cup_stadia.ipynb).

<a title="By Charlie Kritschmar (WMDE) [CC0], from Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Wikidata_stamp.png"><img width="128" alt="Wikidata stamp" src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Wikidata_stamp.png/512px-Wikidata_stamp.png"></a>